In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
data = pd.read_csv('../input/news-extracted-dataset/News_category_extract_dataset.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)
data.shape

In [ ]:
data = data.sample(frac=0.2, random_state=99).reset_index(drop=True)
data = shuffle(data, random_state=22)
data = data.reset_index(drop=True)
data.shape

In [ ]:
from keras.preprocessing.text import Tokenizer
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 200
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['clean_text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(data['clean_text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
Y = pd.get_dummies(data['category']).values
print('Shape of label tensor:', Y.shape)

In [ ]:
data['category'][:5]

In [ ]:
d={0:'ENTERTAINMENT',1:'POLITICS',2:'TRAVEL',3:'WELLNESS'}

In [ ]:
Y[:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
from keras.callbacks import EarlyStopping
epochs = 20
batch_size = 64
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
tokens=list(word_index.keys())
values=list(word_index.values())
vocabulary=dict(zip(values,tokens))

In [ ]:
def convertor(sequence):   
    converted_text_list=[]
    for val in sequence:
        if val!=0:
            converted_text_list.append(vocabulary[val])
    converted_text=' '.join(converted_text_list)
    return converted_text

In [ ]:
idx=6
y_preds = model.predict(X_test)
print(X_test[idx])
print(convertor(X_test[idx]))
print('Actual_class:',d[np.argmax(Y_test[idx])],Y_test[idx])
print('pred_class:',d[np.argmax(y_preds[idx])],y_preds[idx])
classes=list(d.values())

In [ ]:
def new_predict(text):
    X = tokenizer.texts_to_sequences(text)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    return model.predict(X)

In [ ]:
convertor(X_test[idx])

In [ ]:
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=classes)
exp = explainer.explain_instance(convertor(X_test[idx]), new_predict, num_features=20,labels=[0,1,2,3])
#print ('Explanation for class %s' % classes[0])
print ('\n'.join(map(str, exp.as_list())))

In [ ]:
exp = explainer.explain_instance(convertor(X_test[idx]), new_predict, num_features=20, top_labels=2)
print(exp.available_labels())

In [ ]:
exp.show_in_notebook(text=convertor(X_test[idx]))

In [ ]:
import matplotlib.pyplot as plt
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
y_rf_1_pred_test=model.predict(X_test)
y_rf_1_pred_train=model.predict(X_train) 
y_rf_1_pred_test

In [ ]:
labels1=[]
for i in range(len(y_rf_1_pred_test)):
    labels = ['business','entertainment','politics','sport','tech']
    #y_rf_1_pred_test1,labels[np.argmax(y_rf_1_pred_test)]
    labels1.append(labels[np.argmax(y_rf_1_pred_test[i])])
    #print(y_rf_1_pred_test[i],labels1[i])

In [ ]:
labels1

In [ ]:
Y_test1=[]
for i in range(len(y_rf_1_pred_test)):
    labels = ['business','entertainment','politics','sport','tech']
    Y_test1.append(labels[np.argmax(Y_test[i])])


In [ ]:
Y_test1

In [ ]:
from sklearn.metrics import classification_report
#print(classification_report(y_train,y_rf_1_pred_train))
print(classification_report(Y_test1,labels1))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
cm_nb_model1 =multilabel_confusion_matrix(Y_test1, labels1,labels=['business','entertainment','politics','sport','tech'])

In [ ]:
cm_nb_model1